In [55]:
import numpy as np
import pandas as pd
import re
RANDOM_SEED = 7382

In [56]:
tweets_df = pd.read_csv('Twibot-20/tweets_explode.csv')
print(tweets_df.shape)
tweets_df.dropna(inplace=True)
tweets_df.sample(10)

(1999868, 3)


,ID,tweet,label
418699,868485978,RT @Backwards_Dog: Have you heard about #CBD A...,1
823932,2403204333,RT @HughesDirect: Last chance to enter our lat...,1
331263,28884655,RT @WhiteHouse: President @realDonaldTrump is ...,1
842396,1209111705143435264,2-4 Julio '74. Buenos Aires. 1) El Presidente ...,0
1309499,542809663,RT @anothergaze: Another Gaze 03 is here in th...,1
860375,16186995,RT @moment: 8.18.20.\nhttps://t.co/EEAKvUWaIU\n,0
1147092,268363666,RT @Rors7: My wife lost a red iPod around the ...,1
804631,17055538,Musicbed is giving away $50k in film gear! Ent...,1
220392,58913261,"""Abus, combines et privilèges : les dessous du...",0
595811,1009851315869102080,@EAMaddenNFL why isnt MY quarterback cant pass...,1


In [57]:
def extract_num_mentions(tweet, unique=False):
    mentions = re.findall(r'@\w+', tweet)
    if unique:
        return len(set(mentions))
    else:
        return len(mentions)

In [58]:
def is_retweet(tweet):
    return 1 if tweet.startswith('RT') else 0

In [100]:
# using regex expression from https://www.geeksforgeeks.org/extract-urls-present-in-a-given-string/
def extract_num_links(tweet, unique=False):
    # remove comma or peroid as they throw off the regex
    links = re.findall(r'\b((?:https?|ftp|file)://[-a-zA-Z0-9+&@#/%?=~_|!:, .;]*[-a-zA-Z0-9+&@#/%=~_|])', tweet)
    if links:
        links = [link.split() for link in links][0]
        links = [link for link in links if (link.startswith('http') or link.startswith('ftp') or link.startswith('file'))]
    if unique:
        return len(set(links))
    else:
        return len(links)

In [101]:
for tweet in tweets_df['tweet'].sample(10):
    print(extract_num_links(tweet))

0
1
1
1
0
1
1
0
1
1


In [102]:
tweets_df.loc[:, 'num_mentions'] = tweets_df['tweet'].apply(extract_num_mentions)
tweets_df.loc[:, 'num_links'] = tweets_df['tweet'].apply(extract_num_links)
tweets_df.loc[:, 'is_retweet'] = tweets_df['tweet'].apply(is_retweet)

In [103]:
tweets_df.sample(10)

,ID,tweet,label,num_mentions,num_links,is_retweet
1192147,3121978130,Pros and Reddit are like an old married couple...,0,0,1,0
902456,1539474614,"Annem bankada uzun bir süre sırada bekleyip ,...",0,0,0,0
1923670,4255361,"The story of how NotPetya hit Maersk, captured...",0,0,1,0
949080,2795278230,RT @1nicolehall: Nice flex D 💪🏼 @DallinHHall 👀...,1,7,1,1
1419969,65148180,RT @thejulianbass: if y’all can retweet this e...,0,1,1,1
1465289,23233615,RT @Pantera: Happy 52nd Birthday @philiphansel...,1,2,1,1
686349,17348525,Take a virtual class with a State Fair butter ...,0,0,2,0
1226871,15698349,RT @WhiteHouse: Our nation's drug problem will...,1,1,0,1
1811604,268065677,@Wardy5632 @tedwheeler @PortlandPolice Actuall...,1,3,0,0
453037,773504832,@Jamisondontwant Time well spent!\n,1,1,0,0


In [104]:
tweets_df.sort_values(['num_links'], ascending=False)

,ID,tweet,label,num_mentions,num_links,is_retweet
250309,14506253,The 3rd ep of #TheDarkestTimeline podcast is u...,0,2,5,0
1708994,3518971516,RT @AliceeBrookees: https://t.co/YIyGvVMR9r ht...,1,1,5,1
1435356,3408032129,"@Liqui_Exchange @enjin Out on liqui, get tradi...",1,2,5,0
250290,14506253,The 5th ep of #TheDarkestTimelinePodcast calle...,0,1,5,0
209191,1142942118056357888,This is why for securing data and email and me...,1,0,5,0
...,...,...,...,...,...,...
863252,53972789,RT @LordAshcroft: I nearly died from septic sh...,1,1,0,1
863251,53972789,@SethAbramson What did you think of the Hardin...,1,1,0,0
863249,53972789,@paddycosgrave Thanks Paddy\n,1,1,0,0
863248,53972789,@paddycosgrave Probably because ordinary surgi...,1,1,0,0


In [107]:
print(extract_num_links(tweets_df.loc[250309, 'tweet']))
tweets_df.loc[250309, 'tweet']

5


'The 3rd ep of #TheDarkestTimeline podcast is up! And good news—-@kenjeong is finally funny in this one. Congrats Ken!! Also @dannypudi drops in. Available all these places: https://t.co/Bh8F250yV9   https://t.co/Hy2xF0Y4aB https://t.co/BFvpizMKK3 https://t.co/hbgu9DiXtR https://t.co/ifwMfQisa2\n'

In [111]:
user_level = tweets_df[['ID', 'num_mentions', 'num_links', 'is_retweet']].groupby(['ID']).mean().reset_index(drop=True)
user_level.sample(10)

,num_mentions,num_links,is_retweet
9530,0.747475,0.606061,0.252525
10250,0.230769,0.384615,0.000000
3552,1.780000,0.600000,0.615000
697,0.545000,1.285000,0.055000
11699,0.538462,1.461538,0.051282
4541,1.265000,0.655000,0.085000
11093,0.571429,0.821429,0.142857
2148,0.935000,0.495000,0.650000
10972,1.310000,0.015000,0.020000
2454,1.245000,0.275000,0.955000
